In [ ]:
# note that this is a version to run it on colab

In [6]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [4]:
import sys
sys.path.insert(0,'/content/gdrive/MyDrive/CIS6200ProjectWork/')
sys.path.append("/content/gdrive/MyDrive/CIS6200ProjectWork/DiffFoley/inference/")

In [20]:
import os
sys.path.append("/".join(os.getcwd().split("/")[:-1]))

In [7]:
%cd gdrive/MyDrive/CIS6200ProjectWork

/content/gdrive/MyDrive/CIS6200ProjectWork


In [29]:
# have to use this version of mmcv for it to work
! pip install mmcv==1.7.1 -f https://download.openmmlab.com/mmcv/dist/cu121/torch2.0/index.html

Looking in links: https://download.openmmlab.com/mmcv/dist/cu121/torch2.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.4/605.4 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 25.5 MB/s eta 0:00:00
  Created wheel for mmcv: filename=mmcv-1.7.1-py2.py3-none-any.whl size=930716 sha256=87eb94f6bbab4457c6367d50d70b14fdfe64986c25492aabe690e839b50da139
  Stored in directory: /root/.cache/pip/wheels/f8/f9/e3/7e8ab9b98ee9e91fcb677bea6a1ca23b755ddf87a982216acf
Successfully built mmcv


In [ ]:
!pip install omegaconf

In [ ]:
# already done
# ! git clone https://github.com/luosiallen/Diff-Foley.git

Cloning into 'Diff-Foley'...
remote: Enumerating objects: 451, done.
remote: Counting objects: 100% (451/451), done.
remote: Compressing objects: 100% (347/347), done.
remote: Total 451 (delta 134), reused 367 (delta 85), pack-reused 0
Receiving objects: 100% (451/451), 5.27 MiB | 12.72 MiB/s, done.
Resolving deltas: 100% (134/134), done.
Updating files: 100% (330/330), done.


Testing CAVP

In [2]:
import torch
from omegaconf import OmegaConf
import librosa
from IPython.display import Audio

In [ ]:
# you can import code as modules
from DiffFoley.data_preprocess import wav2spec
from DiffFoley.inference.model import cavp_modules
from DiffFoley.inference.model import cavp_model

In [10]:
audio_path = "wav_inputs/verano.wav"

In [65]:
# the length parameter here depends on the input
# it also changes the spectrum shape in the end
y1, spec1 = wav2spec.get_spectrogram(audio_path, 80000)

In [67]:
# ensure that we can replay the audio
Audio(y1, rate = 16000)

In [69]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [70]:
import importlib
def get_obj_from_str(string, reload=False):
    module, cls = string.rsplit(".", 1)
    print(module)
    print(cls)
    print(string)
    if reload:
        module_imp = importlib.import_module(module)
        importlib.reload(module_imp)
    return getattr(importlib.import_module(module, package=None), cls)

def instantiate_from_config(config):
    if not "target" in config:
        if config == '__is_first_stage__':
            return None
        elif config == "__is_unconditional__":
            return None
        raise KeyError("Expected key `target` to instantiate.")
    return get_obj_from_str(config["target"])(**config.get("params", dict()))

In [71]:
config_path = "DiffFoley/inference/config/Stage1_CAVP.yaml"

In [72]:
config = OmegaConf.load(config_path)

In [73]:
cavpmodel = instantiate_from_config(config.model).to(device)

model.cavp_model
CAVP_Inference
model.cavp_model.CAVP_Inference


In [80]:
# appears audio must be a certain length to pool (bc pool is over 16)
test = cavpmodel.encode_spec(torch.tensor(spec1).unsqueeze(0), pool = True)


In [ ]:
# todo: integrate with the rest of tempotokens